In [33]:
import numpy as np
import pandas as pd
np.random.seed(42)
%matplotlib inline

In [46]:
mu_vecl = np.array([0, 0, 0])
cov_mat1 = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
class1_sample = np.random.multivariate_normal(mu_vecl, cov_mat1, 20).T

In [47]:
mu_vec2 = np.array([1, 1, 1])
cov_mat2 = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
class2_sample = np.random.multivariate_normal(mu_vec2, cov_mat2, 20).T

In [48]:
np.cov(class1_sample)

array([[ 0.80014689, -0.14243111,  0.24182031],
       [-0.14243111,  0.52985133, -0.2756814 ],
       [ 0.24182031, -0.2756814 ,  0.98303006]])

In [49]:
from sklearn.decomposition import PCA
sklearn_pca = PCA(n_components=2)
sklearn_transf = sklearn_pca.fit_transform(all_samples.T)